# Toán UDTK | Project 2: Gram – Schmidt
**Sinh viên thực hiện:**   

**MSSV: 23120262**

**MSSV: Tống Dương Thái Hòa**

**Giáo viên hướng dẫn:**  

CN. Võ Nam Thục Đoan

ThS. Trần Hà Sơn

ThS. Nguyễn Hữu Toàn

Lê Trọng Anh Tú

---

## Đề bài

1) Cho A là ma trận có thể phân rã QR. Sinh viên viết chương trình in ra ma trận Q và R, biết
rằng A = QR. 

2) Mở rộng: 

    • Tìm hiểu hàm/ phương thức tương ứng của các thư viện và thực hiện nó, so sánh kết
quả. 

    • Tìm hiểu và trình bày ứng dụng của QR decomposition. 

### Thuật giải Gram-Schmidt

**Input:** Họ các vector $u_1, u_2, \ldots, u_n$ cùng kích thước.

**Output:** 
- Thông báo nếu họ không độc lập tuyến tính
- Ngược lại, trả về:
  - Họ trực giao $\{v_1, v_2, \ldots, v_n\}$ hoặc
  - Họ trực chuẩn $\{q_1, q_2, \ldots, q_n\}$ (là cơ sở của $\text{Span}\{v_1, v_2, \ldots, v_n\}$)

---

**Bước 1:**  
$v_1 = u_1$  
*Nếu $v_1 = \mathbf{0}$: thông báo họ không độc lập tuyến tính và kết thúc.*

**Bước 2:**  
$v_2 = u_2 - \frac{\langle u_2, v_1 \rangle}{\|v_1\|^2}v_1$  
*Nếu $v_2 = \mathbf{0}$: thông báo họ không độc lập tuyến tính và kết thúc.*

**Bước 3:**  
$v_3 = u_3 - \frac{\langle u_3, v_1 \rangle}{\|v_1\|^2}v_1 - \frac{\langle u_3, v_2 \rangle}{\|v_2\|^2}v_2$  
*Nếu $v_3 = \mathbf{0}$: thông báo họ không độc lập tuyến tính và kết thúc.*

$\vdots$

**Bước n:**  
$v_n = u_n - \sum_{i=1}^{n-1}\frac{\langle u_n, v_i \rangle}{\|v_i\|^2}v_i$  
*Nếu $v_n = \mathbf{0}$: thông báo họ không độc lập tuyến tính và kết thúc.*

---

**Bước chuẩn hóa (nếu cần họ trực chuẩn):**

$$
q_1 = \frac{v_1}{\|v_1\|}, \quad 
q_2 = \frac{v_2}{\|v_2\|}, \quad 
\ldots, \quad 
q_n = \frac{v_n}{\|v_n\|}
$$

Trong đó:
- $\langle \cdot, \cdot \rangle$ là tích vô hướng
- $\|\cdot\|$ là chuẩn (độ dài) vector
- $\mathbf{0}$ ký hiệu vector không

### Định lý phân rã QR
Cho ma trận $A \in \mathbb{R}^{m \times n}$ (với $m \geq n$) có $n$ vector cột độc lập tuyến tính. Khi đó, tồn tại duy nhất:

$$ A = QR $$

với:
- $Q \in \mathbb{R}^{m \times n}$: Ma trận có các cột trực chuẩn ($Q^\top Q = I_n$)
- $R \in \mathbb{R}^{n \times n}$: Ma trận tam giác trên khả nghịch

### Thuật toán phân rã QR bằng Gram-Schmidt

#### Input
- Ma trận $A = [\mathbf{a}_1 | \mathbf{a}_2 | \cdots | \mathbf{a}_n] \in \mathbb{R}^{m \times n}$

#### Output
- Nếu các cột của $A$ phụ thuộc tuyến tính → Thông báo lỗi
- Ngược lại → Trả về cặp ma trận $(Q, R)$ thỏa mãn $A = QR$

#### Các bước thực hiện

1. **Trích xuất các vector cột**  
   Xác định $\{\mathbf{a}_1, \mathbf{a}_2, \ldots, \mathbf{a}_n\}$ là các vector cột của $A$

2. **Trực chuẩn hóa Gram-Schmidt**  
   Áp dụng quá trình Gram-Schmidt để thu được hệ trực chuẩn $\{\mathbf{q}_1, \mathbf{q}_2, \ldots, \mathbf{q}_n\}$:
   - Nếu gặp vector $\mathbf{0}$ trong quá trình → Dừng và báo lỗi

3. **Xây dựng ma trận Q**  
   $$ Q = [\mathbf{q}_1 | \mathbf{q}_2 | \cdots | \mathbf{q}_n] $$

4. **Tính ma trận R**  
   Ma trận tam giác trên $R$ được tính bởi:
   $$
   R = \begin{bmatrix}
   r_{11} & r_{12} & \cdots & r_{1n} \\
   0 & r_{22} & \cdots & r_{2n} \\
   \vdots & \vdots & \ddots & \vdots \\
   0 & 0 & \cdots & r_{nn}
   \end{bmatrix}
   $$
   với:
   - $r_{ij} = \langle \mathbf{a}_j, \mathbf{q}_i \rangle$ khi $i \leq j$
   - $r_{ij} = 0$ khi $i > j$

5. **Kiểm tra và trả kết quả**  
   Xác nhận $A = QR$ và trả về bộ phân tích

In [1]:
import numpy as np

def norm(vector):
    squared_sum = sum(x**2 for x in vector)
    return squared_sum**0.5

def dot_product(v1, v2):
    return sum(x * y for x, y in zip(v1, v2))

def vector_subtract(v1, v2):
    return [x - y for x, y in zip(v1, v2)]

def gram_schmidt_qr_decomposition(A):
    A = np.array(A, dtype=float)
    m, n = A.shape
    
    # Khởi tạo ma trận Q và R
    Q = np.zeros((m, n))
    R = np.zeros((n, n))
    
    for j in range(n):
        # Bắt đầu với vector cột thứ j của A
        v = A[:, j].copy()
        
        # Trừ đi hình chiếu lên các vector trước đó
        for i in range(j):
            R[i, j] = dot_product(Q[:, i], A[:, j])
            v = vector_subtract(v, R[i, j] * Q[:, i])
        
        # Kiểm tra vector có phải vector 0 không
        if np.allclose(v, np.zeros_like(v)):
            print("Các cột của ma trận không độc lập tuyến tính")
            return None, None
        
        # Tính norm và chuẩn hóa
        R[j, j] = norm(v)
        Q[:, j] = v / R[j, j]
    
    # Đặt chế độ hiển thị để tránh ký hiệu khoa học
    np.set_printoptions(suppress=True)
    
    return Q, R

# Một số test cases

## Test case 1: Ma trận vuông 3 x 3

In [2]:
A = np.array([[1, 0, 0],
              [1, 1, 0],
              [1, 1, 1]])

Q, R = gram_schmidt_qr_decomposition(A)
print("Ma trận Q:")
print(Q)
print("\nMa trận R:")
print(R)

Ma trận Q:
[[ 0.57735027 -0.81649658 -0.        ]
 [ 0.57735027  0.40824829 -0.70710678]
 [ 0.57735027  0.40824829  0.70710678]]

Ma trận R:
[[1.73205081 1.15470054 0.57735027]
 [0.         0.81649658 0.40824829]
 [0.         0.         0.70710678]]


## Test case 2: Ma trận 3 x 1

In [3]:
B = np.array([[0.87695113],
              [0.3284933],
              [-0.35078323]])

Q, R = gram_schmidt_qr_decomposition(B)
print("Ma trận Q:")
print(Q)
print("\nMa trận R:")
print(R)

Ma trận Q:
[[ 0.87695113]
 [ 0.3284933 ]
 [-0.35078323]]

Ma trận R:
[[1.]]


## Test case 3: Ma trận 3 x 4

In [4]:
C = np.array([[1, 0, 0, 1],
              [1, 1, 0, 0],
              [1, 1, 1, 1]])

Q, R = gram_schmidt_qr_decomposition(C)
print("Ma trận Q:")
print(Q)
print("\nMa trận R:")
print(R)

Các cột của ma trận không độc lập tuyến tính
Ma trận Q:
None

Ma trận R:
None


## Kiểm tra phân rã QR

Sau khi thực hiện phân rã QR, ta có thể kiểm tra tính chính xác bằng cách nhân lại các ma trận $Q$ và $R$:

### Phương pháp kiểm tra
1. Thực hiện phép nhân ma trận:  
   $$ A_{\text{reconstructed}} = Q \times R $$

2. So sánh với ma trận gốc $A$:
   $$ \|A - QR\|_F < \epsilon $$
   với $\epsilon$ là ngưỡng sai số chấp nhận được (thường chọn $\epsilon = 10^{-10}$)

In [5]:
Q, R = gram_schmidt_qr_decomposition(A)

result = np.dot(Q, R)
result

array([[ 1.,  0.,  0.],
       [ 1.,  1., -0.],
       [ 1.,  1.,  1.]])

### Hàm/phương thức tương ứng của các thư viện

### 1. numpy.linalg.qr()

### Test case 1: Ma trận vuông 3 x 3

In [6]:
from numpy.linalg import qr as qr_numpy

A = np.array([[1, 0, 0],
              [1, 1, 0],
              [1, 1, 1]])

Q, R = qr_numpy(A)

print("Ma trận Q_numpy:")
print(Q)
print("\nMa trận R_numpy:")
print(R)


Ma trận Q_numpy:
[[-0.57735027  0.81649658 -0.        ]
 [-0.57735027 -0.40824829 -0.70710678]
 [-0.57735027 -0.40824829  0.70710678]]

Ma trận R_numpy:
[[-1.73205081 -1.15470054 -0.57735027]
 [ 0.         -0.81649658 -0.40824829]
 [ 0.          0.          0.70710678]]


### Test case 2: Ma trận 1 x 3

In [7]:
B = np.array([[ 0.87695113],
              [ 0.3284933 ],
              [-0.35078323]])

# Phân rã QR của ma trận B
Q, R = qr_numpy(B)

# In ra ma trận Q và R
print("Ma trận Q_numpy:")
print(Q)
print("\nMa trận R_numpy:")
print(R)

Ma trận Q_numpy:
[[-0.87695113]
 [-0.3284933 ]
 [ 0.35078323]]

Ma trận R_numpy:
[[-1.]]


### Test case 3: Ma trận 4 x 3

In [8]:
C = np.array([[1, 0, 0, 1],
              [1, 1, 0, 0],
              [1, 1, 1, 1]])

# Phân rã QR của ma trận C
Q, R = qr_numpy(C)

# In ra ma trận Q và R
print("Ma trận Q_numpy:")
print(Q)
print("\nMa trận R_numpy:")
print(R)

Ma trận Q_numpy:
[[-0.57735027  0.81649658 -0.        ]
 [-0.57735027 -0.40824829 -0.70710678]
 [-0.57735027 -0.40824829  0.70710678]]

Ma trận R_numpy:
[[-1.73205081 -1.15470054 -0.57735027 -1.15470054]
 [ 0.         -0.81649658 -0.40824829  0.40824829]
 [ 0.          0.          0.70710678  0.70710678]]


### 2. scipy.linalg.qr()

### Test case 1: Ma trận vuông 3 x 3

In [9]:
from scipy.linalg import qr as qr_scipy

A = np.array([[1, 0, 0],
              [1, 1, 0],
              [1, 1, 1]])

Q, R = qr_scipy(A)

print("Ma trận Q_scipy:")
print(Q)
print("\nMa trận R_scipy:")
print(R)

Ma trận Q_scipy:
[[-0.57735027  0.81649658 -0.        ]
 [-0.57735027 -0.40824829 -0.70710678]
 [-0.57735027 -0.40824829  0.70710678]]

Ma trận R_scipy:
[[-1.73205081 -1.15470054 -0.57735027]
 [ 0.         -0.81649658 -0.40824829]
 [ 0.          0.          0.70710678]]


### Test case 2: Ma trận 1 x 3

In [10]:
B = np.array([[ 0.87695113],
              [ 0.3284933 ],
              [-0.35078323]])

# Phân rã QR của ma trận B
Q, R = qr_scipy(B)

# In ra ma trận Q và R
print("Ma trận Q_scipy:")
print(Q)
print("\nMa trận R_scipy:")
print(R)

Ma trận Q_scipy:
[[-0.87695113 -0.3284933   0.35078323]
 [-0.3284933   0.94250897  0.06139208]
 [ 0.35078323  0.06139208  0.93444215]]

Ma trận R_scipy:
[[-1.]
 [ 0.]
 [ 0.]]


### Test case 3: Ma trận 4 x 3

In [11]:
C = np.array([[1, 0, 0, 1],
              [1, 1, 0, 0],
              [1, 1, 1, 1]])

# Phân rã QR của ma trận C
Q, R = qr_scipy(C)

# In ra ma trận Q và R
print("Ma trận Q_numpy:")
print(Q)
print("\nMa trận R_numpy:")
print(R)

Ma trận Q_numpy:
[[-0.57735027  0.81649658 -0.        ]
 [-0.57735027 -0.40824829 -0.70710678]
 [-0.57735027 -0.40824829  0.70710678]]

Ma trận R_numpy:
[[-1.73205081 -1.15470054 -0.57735027 -1.15470054]
 [ 0.         -0.81649658 -0.40824829  0.40824829]
 [ 0.          0.          0.70710678  0.70710678]]


### So sánh giữa hai hàm numpy.linalg.qr() và scipy.linalg.qr() 

| Đặc điểm               | numpy.linalg.qr()                | scipy.linalg.qr()                |
|------------------------|----------------------------------|----------------------------------|
| **Thư viện**           | NumPy                            | SciPy                            |
| **Thuật toán**         | Householder reflections          | Householder/Givens rotations     |
| **Đầu vào**            | Mảng NumPy                       | Mảng NumPy, ma trận thưa         |
| **Mode mặc định**      | 'reduced'                        | 'full'                           |
| **Xử lý ma trận suy biến** | Cảnh báo                      | Hỗ trợ pivoting                  |
| **Hiệu năng**          | Tốt cho ma trận nhỏ              | Tối ưu cho ma trận lớn           |
 

---

## So sánh kết quả theo từng test case giữa hàm tự xây dựng và 2 hàm sử dụng thư viện

1. **Test case 1 (ma trận 3x3)**: Kết quả trả về của ma trận Q, R của hàm tự xây dựng **khác** về **dấu** so với 2 hàm sử dụng thư viện

2. **Test case 2 (ma trận 1x3)**:
   - Kết quả trả về của ma trận Q, R của 3 hàm là **giống nhau**, chỉ khác về cách trình bày kết quả
   - Đối với hàm `numpy.linalg.qr()` thì mode trả về mặc định là reduced (dưới dạng rút gọn). Trong khi đó với hàm `scipy.linalg.qr()` thì mode trả về mặc định là full (dưới dạng đầy đủ)
   - Để có thể lấy được kết quả trả về của hai hàm là như nhau, ta có thể chỉnh:
     - `mode = 'economic'` cho `scipy.linalg.qr()`
     - Hoặc `mode = 'complete'` cho `numpy.linalg.qr()`

3. **Test case 3 (ma trận 4x3)**: Kết quả trả về của ma trận Q, R của hàm tự xây dựng **khác biệt** rõ rệt so với hai hàm sử dụng thư viện

## Giải thích về sự khác nhau của hàm tự xây dựng và hàm thư viện

1. **Hàm gram_schmidt_qr_decomposition(A)**:
   - Sử dụng thuật giải **Gram-Schmidt**
   - Sử dụng phương pháp lặp để tạo ra các vector cột của ma trận Q và tính toán ma trận R
   - Trong quá trình lặp, các phép toán tích vô hướng và trừ vector được sử dụng
   - Phương pháp Gram-Schmidt có thể không hoạt động tốt với ma trận có các cột gần tương đương hoặc gần tương đương tuyến tính

2. **Hàm sử dụng thư viện numpy.linalg.qr()**:
   - Sử dụng phương pháp tối ưu hóa và triển khai hiệu quả hơn
   - Sử dụng phép biến đổi **Householder** và/hoặc **phép biến đổi Givens**
   - Được tối ưu hóa để xử lý hiệu quả các ma trận lớn và ma trận có cột gần tương đương

=> Vì cách tiếp cận và thuật toán khác nhau, kết quả trả về có thể khác nhau. Khi thực hiện phân rã QR, một ma trận có thể có nhiều phân rã khác nhau, nhưng các phân rã đều thỏa mãn tính chất cơ bản của phân rã QR.

Cụ thể, nếu $A$ là một ma trận có kích thước $m \times n$ $(m >= n)$, phân rã $QR$ của $A$ là sự phân tách $A$ thành hai ma trận $Q$ và $R$, sao cho:

- Ma trận Q là một ma trận Orthogonal (trực giao), có kích thước $m \times m$ và thỏa mãn $Q^T \cdot Q = I$
- Ma trận $R$ là một ma trận Upper Triangular (tam giác trên), có kích thước $n \times n$

## Một số ứng dụng của QR decomposition

Phân rã QR (QR decomposition) là một phương pháp quan trọng trong đại số tuyến tính và có nhiều ứng dụng:

1. **Giải hệ phương trình tuyến tính**: Sử dụng phân rã QR để giải các hệ phương trình tuyến tính dạng $Ax = b$, trong đó $A$ là ma trận hệ số, $x$ là vector ẩn cần tìm, và $b$ là vector kết quả.

2. **Tính toán ma trận nghịch đảo**: Phân rã QR hỗ trợ tính toán ma trận nghịch đảo của một ma trận vuông $A$ bằng cách sử dụng các ma trận $Q$ và $R$.

3. **Phân tích giá trị riêng**: QR decomposition là một bước quan trọng trong thuật toán QR để tính toán giá trị riêng của ma trận.

4. **Phân rã SVD (Singular Value Decomposition)**: Phân rã QR được sử dụng như một bước trung gian trong việc tính toán phân rã giá trị kỳ dị (SVD) của ma trận.

5. **Tính toán least squares**: Phân rã QR được sử dụng để giải bài toán tối ưu hóa least squares, đặc biệt khi ma trận $A$ không vuông hoặc suy biến.

6. **Phân tích suy biến**: Hỗ trợ phân tích và xử lý các ma trận suy biến hoặc gần suy biến, giúp cải thiện độ ổn định số học trong các bài toán đại số tuyến tính.

## Ứng dụng trong hệ thống Multiple-Input and Multiple-Output

Phân rã QR được sử dụng rộng rãi trong hệ thống MIMO:

1. **Beamforming:** Sử dụng phân rã QR để tạo ra các ma trận beamforming, trong đó ma trận Q chứa các vector beamforming trực giao và ma trận R chứa thông tin về hướng beam.

2. **MIMO Detection:** Phân rã QR hỗ trợ giải mã tín hiệu từ các anten thu, giúp ước lượng tín hiệu từ các anten truyền và đưa ra quyết định về thông tin truyền gửi.

3. **Channel Equalization:** Sử dụng phân rã QR để cân bằng kênh truyền giữa các anten, tối ưu hóa việc truyền tín hiệu.

4. **Precoding:** Tối ưu hóa việc truyền tín hiệu từ các anten truyền bằng cách sử dụng ma trận precoding được tạo từ phân rã QR.

5. **MIMO Channel Estimation:** Ước lượng kênh truyền giữa các anten bằng cách sử dụng phân rã QR trên tín hiệu nhận được, giúp tối ưu hóa việc truyền tín hiệu trong hệ thống MIMO.

## Ý tưởng thực hiện và mô tả các hàm trong hàm QR decomposition tự xây dựng

### Các hàm phụ hỗ trợ cho hàm chính

1. **Hàm norm(vector)**: Tính toán độ dài (norm) của một vector
2. **Hàm dot_product(v1, v2)**: Tính tích vô hướng của hai vector
3. **Hàm vector_subtract(v1, v2)**: Trừ hai vector

### Hàm chính gram_schmidt_qr_decomposition()

Ý tưởng chính:
- Thực hiện phân rã QR của ma trận A bằng phương pháp Gram-Schmidt
- Chia ma trận A thành hai ma trận Q (trực giao) và R (tam giác trên)

Mô tả chi tiết:
1. Khởi tạo ma trận Q và R với giá trị 0
2. Lặp qua từng cột j trong ma trận A:
   - Lấy vector cột j của A
   - Lặp qua từng cột i từ 0 đến j-1 trong Q:
     - Tính R[i, j] bằng tích vô hướng
     - Trừ đi phần tử R[i, j] nhân với cột i của Q
   - Tính R[j, j] bằng độ dài của vector
   - Chuẩn hóa vector để có cột j của Q
3. Trả về ma trận Q và R

*Hai hàm `np.set_printoptions(suppress=True)` và `np.zeros()` là hàm của thư viện NumPy để thiết lập tùy chọn hiển thị và tạo ma trận zeros.*